In [1]:
import os

In [2]:
%pwd

'e:\\Code\\NLP_Text_Summarize_Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Code\\NLP_Text_Summarize_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationModelConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [6]:
from src.Text_Summarize.constants import *
from src.Text_Summarize.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath= PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_evaluation_model_config(self) -> EvaluationModelConfig:
        config = self.config.evaluation_model

        create_directories([config.root_dir])

        evaluation_model_config = EvaluationModelConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )

        return evaluation_model_config


In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

In [10]:
class EvaluationModel:
    def __init__(self, config: EvaluationModelConfig):
        self.config = config

    
    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i: i + batch_size]



    def calculate_metric_on_test_ds(self,dataset,metric,model,tokenizer, batch_size=16
                                    , device="cuda" if torch.cuda.is_available() else "cpu",
                                    column_text="article", column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
                zip(article_batches, target_batches), total=len(article_batches)
        ):

            inputs = tokenizer(article_batch, max_lenght=1024, truncation=True, padding="max_lenght", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device), attention_mask=inputs["attention_mask"].to(device),
                                        length_penalty=0.8, num_beams=8, max_length=128)
            
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,clean_up_tokenization_spaces=True)
                                for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        score = metric.compute()
        return score





    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        dataset_corpus_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1","rouge2","rougeL","rougeLsum"]
        rouge_metrics = load_metric("rouge")

        score = self.calculate_metric_on_test_ds(
            dataset_corpus_pt['test'][0:10], rouge_metrics, model_pegasus, tokenizer, batch_size=2, column_text='dialogue', column_summary='summary'
        )

        rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

        df= pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config_metric_file_name, index=False)


In [11]:
try:
    config = ConfigurationManager()
    evaluation_model_config = config.get_evaluation_model_config()
    evaluation_model_config = EvaluationModel(config=evaluation_model_config)
    evaluation_model_config.evaluate()
except Exception as e:
    raise e

[2023-05-18 20:14:03,142: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-05-18 20:14:03,151: INFO: common: yaml file: params.yaml loaded successfully]
[2023-05-18 20:14:03,153: INFO: common: created directory at: artifacts]
[2023-05-18 20:14:03,155: INFO: common: created directory at: artifacts/model_evaluation]


C:\Users\bokti\AppData\Local\Temp\ipykernel_25280\2973449339.py:59: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
100%|██████████| 5/5 [03:54<00:00, 46.91s/it]

[2023-05-18 20:18:18,394: INFO: rouge_scorer: Using default tokenizer.]
